In [1]:
%pip install pandas torch transformers beir scikit-learn scipy faiss-cpu tensorboard pyarrow fastparquet datasets
import torch

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 29.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB 1.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 35.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 KB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 34.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 17.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 23.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 40.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 14.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 40.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

CUDA available: True
GPU: NVIDIA A30
GPU Memory: 25.3 GB


In [5]:
#Tokenize data
#Place datasets under contriever/scripts/preprocess/
!bash ./scripts/preprocess/tokenization_script.sh

./wikipedia_en_20231101_subset.txt
Encoding ./tmp-tokenization-bert-base-uncased-wikipedia_en_20231101_subset.txt/000...
Saving at ./encoded-data/bert-base-uncased/wikipedia_en_20231101_subset/000.pkl...
Encoding ./tmp-tokenization-bert-base-uncased-wikipedia_en_20231101_subset.txt/008...
Encoding ./tmp-tokenization-bert-base-uncased-wikipedia_en_20231101_subset.txt/004...
Encoding ./tmp-tokenization-bert-base-uncased-wikipedia_en_20231101_subset.txt/002...
Encoding ./tmp-tokenization-bert-base-uncased-wikipedia_en_20231101_subset.txt/003...
Encoding ./tmp-tokenization-bert-base-uncased-wikipedia_en_20231101_subset.txt/007...
Encoding ./tmp-tokenization-bert-base-uncased-wikipedia_en_20231101_subset.txt/001...
Encoding ./tmp-tokenization-bert-base-uncased-wikipedia_en_20231101_subset.txt/005...
Encoding ./tmp-tokenization-bert-base-uncased-wikipedia_en_20231101_subset.txt/006...
Saving at ./encoded-data/bert-base-uncased/wikipedia_en_20231101_subset/002.pkl...
Saving at ./encoded-data/

In [ ]:
# training the data
# prof mentioned that you should be able to hit 70% accuracy just by training on more data (TA trained for >= 1 day)
!bash /users/vishraj/cs511-fall2025-p3-contriever/scripts/train/contriever.sh

Using device: cuda
                            augmentation: delete                                  	[default: none]
                                   beta1: 0.9                                     
                                   beta2: 0.98                                    
                            chunk_length: 256                                     
                       continue_training: False                                   
                        contrastive_mode: moco                                    
                                 dropout: 0.1                                     
                                     eps: 1e-06                                   
                               eval_data: []                                      
                           eval_datasets: []                                      
                       eval_datasets_dir: ./                                      
                               eval_freq: 500       

In [ ]:
# if you need to delete a model checkpoint
# !rm -rf ./model/average-rmin0.05-rmax0.5-T0.05-4096-0.9995-bert-base-uncased-delete-0.1

In [1]:
# Evaluate your model using NQ benchmark
#Step1: prepare passage subset
#run under contriever/scripts/evaluation/FiD/open_domain_data
#place psgs_w100.tsv under contriever/scripts/evaluation/FiD/open_domain_data/
# !cp drive/MyDrive/datasets/psgs_w100.tsv drive/MyDrive/contriever/scripts/evaluation/FiD/open_domain_data/

!python3 ./scripts/evaluation/FiD/open_domain_data/select_question_subset.py \
--data_dir ./scripts/evaluation/FiD/open_domain_data/download > ./scripts/evaluation/FiD/open_domain_data/passage.idx

In [4]:
!python3 ./scripts/evaluation/FiD/open_domain_data/create_passage_subset.py \
--passage_idx ./scripts/evaluation/FiD/open_domain_data/passage.idx \
--passages_file ./datasets/psgs_w100.tsv \
--output_file ./scripts/evaluation/FiD/open_domain_data/psgs_w100_subset.tsv \
--seed 42

Loading required passage IDs from ./scripts/evaluation/FiD/open_domain_data/passage.idx...
Loaded 92042 required passage IDs

Target subset size: 210153 passages (1% of 21015324)
Scanning passages file (byte-preserving)...
Reading lines: 100%|█████████| 21015324/21015324 [01:12<00:00, 289145.90lines/s]

Required passages found: 92042
Additional random passages needed: 118111

SUBSET SUMMARY
Total passages in original: 21,015,324
Required IDs: 92,042
Required passages found: 92,042
Final subset size: 210,153
Target (1%): 210,153
Percentage of original: 1.00%

Saving subset to ./scripts/evaluation/FiD/open_domain_data/psgs_w100_subset.tsv (byte-identical lines)...
✓ Saved 210153 lines to ./scripts/evaluation/FiD/open_domain_data/psgs_w100_subset.tsv


In [6]:
#Evaluate your model using NQ benchmark
#Step2: preprocess for testing
!python3 ./scripts/evaluation/FiD/src/preprocess.py \
--num_questions 1000 --passages_file ./scripts/evaluation/FiD/open_domain_data/psgs_w100_subset.tsv


Loading passages from ./scripts/evaluation/FiD/open_domain_data/psgs_w100_subset.tsv
Loading passages: 100%|██████████████| 210154/210154 [00:04<00:00, 44362.46it/s]
Loaded 210153 passages
Selecting test examples


In [9]:
# Step3: use your model to embed passages
!python3 ./scripts/evaluation/generate_passage_embeddings.py  \
--passages ./scripts/evaluation/FiD/open_domain_data/psgs_w100_subset.tsv \
 --model_name_or_path ./model/average-rmin0.05-rmax0.5-T0.05-4096-0.9995-bert-base-uncased-delete-0.1/checkpoint/lastlog/  \
 --output_dir ./scripts/evaluation/contriever_embeddings  \
 --shard_id 0 \
 --num_shards 1

Using device: cuda
Model loaded from ./model/average-rmin0.05-rmax0.5-T0.05-4096-0.9995-bert-base-uncased-delete-0.1/checkpoint/lastlog/.
Embedding generation for 210153 passages from idx 0 to 210153.
Encoding passages: 100%|██████████| 210153/210153 [04:55<00:00, 712.27passage/s]
Saving 210153 passage embeddings to ./scripts/evaluation/contriever_embeddings/passages_00.
Total passages processed 210153. Written to ./scripts/evaluation/contriever_embeddings/passages_00.


In [16]:
# Step4: retrieve the passages
#run under scripts/evaluation/FiD/open_domain_data
!python3 ./scripts/evaluation/passage_retrieval.py \
--model_name_or_path ./model/average-rmin0.05-rmax0.5-T0.05-4096-0.9995-bert-base-uncased-delete-0.1/checkpoint/lastlog/  \
--passages ./scripts/evaluation/FiD/open_domain_data/psgs_w100_subset.tsv  \
--passages_embeddings ./scripts/evaluation/contriever_embeddings/*  \
--data ./open_domain_data/NQ/test.json \
--output_dir ./scripts/evaluation/FiD/open_domain_data/contriever_nq

Using device: cuda
Loading model from: ./model/average-rmin0.05-rmax0.5-T0.05-4096-0.9995-bert-base-uncased-delete-0.1/checkpoint/lastlog/
Indexing passages from files ['./scripts/evaluation/contriever_embeddings/passages_00']
Loading file ./scripts/evaluation/contriever_embeddings/passages_00
Total data indexed 210153
Data indexing completed.
Indexing time: 2.1 s.
loading passages
['./open_domain_data/NQ/test.json']
processing path ./open_domain_data/NQ/test.json
Questions embeddings shape: torch.Size([1000, 768])
Searching for 100 documents in the index
100%|███████████████████████████████████████| 1000/1000 [01:22<00:00, 12.15it/s]
Search time: 82.3 s.
Validation results: top k documents hits %s [50, 82, 114, 130, 148, 169, 183, 202, 219, 231, 241, 252, 261, 267, 275, 289, 295, 308, 312, 322, 326, 331, 335, 341, 343, 349, 352, 356, 363, 368, 371, 373, 381, 388, 393, 396, 402, 403, 407, 410, 414, 414, 419, 422, 424, 427, 430, 431, 436, 441, 444, 444, 448, 448, 449, 450, 452, 454, 457